In [ ]:
from pathlib import Path
import os
import shutil
import numpy as np
import pandas as pd

from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# exp. data
setsizes = [9, 16, 25, 36]
subjects = np.arange(49)

stimulus_dir = 'data/stimuli/'
stimuli = [s for s in os.listdir(stimulus_dir)]

output_dir = 'results/exemplar_choice-screens/'
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

In [ ]:
def make_choice_screen(trial_stimuli, stimulus_positions, stimulus_dir):
    # black background
    screen = Image.new(mode='RGBA', size=(1280, 1024), color='black')
    for i, stimfile in enumerate(trial_stimuli):
        # get stim pos
        stimbox = stimulus_positions.loc[i].values.astype(np.int)
        # get target size
        stimsize = (stimbox[2]-stimbox[0], stimbox[3]-stimbox[1])
        # load stim img
        stimimg = Image.open(stimulus_dir+stimfile)
        # resize stim
        stimimg = stimimg.resize(size=stimsize)
        # paste
        screen.paste(stimimg, box=stimbox)
    return screen

In [ ]:
np.random.seed(1)

for setsize in setsizes:
    # load data
    setsize_data = pd.read_csv('data/summary_files/{}_data.csv'.format(setsize))
    stimulus_positions = pd.read_csv('data/stimulus_positions/{}_stimulus_positions.csv'.format(setsize), header=None)
    stimulus_centers = np.concatenate((
        (stimulus_positions.loc[:,0].values+0.5*(stimulus_positions.loc[:,2].values-stimulus_positions.loc[:,0].values))[:,None],
        (stimulus_positions.loc[:,1].values+0.5*(stimulus_positions.loc[:,3].values-stimulus_positions.loc[:,1].values))[:,None]), axis=1)
    for subject in subjects:
        # subset & load data
        subject_data = setsize_data[setsize_data['subject']==subject].copy()
        subject_gaze_data = pd.read_csv('data/subject_files/{}_{}_fixations.csv'.format(subject, setsize))
        # get trials
        trials = subject_gaze_data.trial.unique()
        for trial in np.unique(trials):
            out_path = output_dir+'setsize-{}_sub-{}_trial-{}.png'.format(setsize, subject, trial)
            if not os.path.isfile(out_path):
                print('Processing: setsize: {}, subject: {}, trial: {}'.format(setsize, subject, trial))
                # subset data
                trial_data = subject_data[subject_data['trial']==trial].copy()
                trial_gaze_data = subject_gaze_data[subject_gaze_data['trial']==trial].copy()
                # get trial stimuli
                trial_stimuli = trial_data[['stimulus_{}'.format(i) for i in range(setsize)]].values[0]
                # make choice screen
                choice_screen = make_choice_screen(trial_stimuli, stimulus_positions, stimulus_dir)
                # make gif & save
                choice_screen.save(out_path)